---
title: "Pruebas t para Muestras Independientes y Test de Mann-Whitney con Python"
subtitle: "Comparando Medias de Dos Grupos Independientes"
format: 
    live-html
pyodide:
  packages:
    - pandas
    - numpy
    - scipy
    - statsmodels
    - seaborn
    - matplotlib
---

## Introducción a la Comparación de Dos Medias

En esta lección, nos enfocamos en comparar las medias de dos grupos independientes, una tarea común en la investigación para determinar si existen diferencias significativas entre dos poblaciones distintas (por ejemplo, un grupo de tratamiento y un grupo de control).

Para esto, utilizamos dos pruebas principales:

-   **Prueba t para dos muestras independientes**: Evalúa si las medias de dos grupos son significativamente diferentes. Asume que los datos se distribuyen normalmente.
-   **Test de Mann-Whitney U (o de Wilcoxon-Mann-Whitney)**: Es la alternativa no paramétrica a la prueba t. Compara las medianas de dos grupos y no requiere normalidad en los datos, ya que trabaja con los rangos.

Ambas pruebas pueden ser vistas como **modelos lineales simples** que buscan determinar si la pertenencia a un grupo tiene un efecto significativo en la variable de resultado.

---

## Puntos Clave

1.  **Prueba t como Modelo Lineal**: El modelo predice el resultado (`y`) basándose en la pertenencia a un grupo. La variable de grupo se codifica numéricamente (ej. 0 y 1), una técnica conocida como **codificación dummy**.
    \[ y = \beta_0 + \beta_1 \cdot \text{grupo} \]
    Donde `β₁` representa la diferencia media entre los dos grupos.

2.  **Test de Mann-Whitney U como Modelo Lineal**: El concepto es similar, pero el modelo se aplica a los **rangos** de la variable de resultado en lugar de a sus valores originales.
    \[ \text{rango}(y) = \beta_0 + \beta_1 \cdot \text{grupo} \]

3.  **Codificación Dummy**: Es el proceso de convertir variables categóricas (como los nombres de los grupos) en variables numéricas (0 y 1) para que puedan ser utilizadas en modelos lineales. Las librerías de Python como `statsmodels` manejan esto automáticamente.

---

## Ejemplo en Python: Comparación de Dos Medias

Realicemos una comparación de dos grupos independientes utilizando la prueba t y el test de Mann-Whitney U en Python.

### Paso 1: Generar y Visualizar los Datos

Creamos un conjunto de datos con dos grupos (`A` y `B`) y visualizamos sus distribuciones.

```{pyodide}
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
import matplotlib.pyplot as plt

# Generar datos para dos grupos independientes
np.random.seed(42)
N = 30
data = pd.DataFrame({
    'valor': np.concatenate([
        np.random.normal(loc=20, scale=5, size=N),
        np.random.normal(loc=25, scale=5, size=N)
    ]),
    'grupo': np.repeat(['A', 'B'], N)
})

# Visualización de los datos
plt.figure(figsize=(8, 6))
sns.boxplot(x='grupo', y='valor', data=data)
sns.stripplot(x='grupo', y='valor', data=data, color='black', jitter=0.2, alpha=0.6)
plt.title('Comparación de los Grupos A y B')
plt.xlabel('Grupo')
plt.ylabel('Valor')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
```

### Paso 2: Realizar las Pruebas Estadísticas

Ahora, aplicamos las pruebas para comparar las medias de los dos grupos.

```{pyodide}
# Separar los datos por grupo
grupo_a = data[data['grupo'] == 'A']['valor']
grupo_b = data[data['grupo'] == 'B']['valor']

# Prueba t para muestras independientes
t_statistic, p_value_ttest = stats.ttest_ind(grupo_a, grupo_b)
print(f"--- Prueba t para Muestras Independientes ---")
print(f"Estadístico t: {t_statistic:.4f}")
print(f"Valor p: {p_value_ttest:.4f}\n")

# Test de Mann-Whitney U
u_statistic, p_value_mann = stats.mannwhitneyu(grupo_a, grupo_b, alternative='two-sided')
print(f"--- Test de Mann-Whitney U ---")
print(f"Estadístico U: {u_statistic:.4f}")
print(f"Valor p: {p_value_mann:.4f}\n")

# Equivalencia con un Modelo Lineal
# Statsmodels crea la codificación dummy automáticamente
modelo = ols('valor ~ grupo', data=data).fit()
print(f"--- Modelo Lineal (Equivalente a la Prueba t) ---")
print(modelo.summary())

```

### Interpretación

-   **Prueba t y Modelo Lineal**: El valor p del coeficiente del grupo en el modelo lineal es idéntico al de la prueba t. El coeficiente para `grupo[T.B]` (5.59) representa la diferencia media entre el grupo B y el grupo A (el de referencia).
-   **Test de Mann-Whitney U**: Proporciona un valor p similar, confirmando que hay una diferencia significativa entre los grupos. Esta prueba es preferible si los datos no son normales.

Ambos enfoques nos llevan a la misma conclusión: existe una diferencia estadísticamente significativa entre los grupos A y B.